<a href="https://colab.research.google.com/github/Asto7/deep-learning/blob/main/brainTumor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle


In [2]:
from google.colab import files


In [3]:
files.upload()

Saving kaggle (1).json to kaggle (1).json


{'kaggle (1).json': b'{"username":"satyanandsharma","key":"da01bd93271217f7c46dcb29a9150c57"}'}

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle datasets download -d sartajbhuvaji/brain-tumor-classification-mri

 97% 84.0M/86.8M [00:00<00:00, 80.1MB/s]
100% 86.8M/86.8M [00:00<00:00, 99.2MB/s]


In [9]:
!unzip ./brain-tumor-classification-mri


Archive:  ./brain-tumor-classification-mri.zip
  inflating: Testing/glioma_tumor/image(1).jpg  
  inflating: Testing/glioma_tumor/image(10).jpg  
  inflating: Testing/glioma_tumor/image(100).jpg  
  inflating: Testing/glioma_tumor/image(11).jpg  
  inflating: Testing/glioma_tumor/image(12).jpg  
  inflating: Testing/glioma_tumor/image(13).jpg  
  inflating: Testing/glioma_tumor/image(14).jpg  
  inflating: Testing/glioma_tumor/image(15).jpg  
  inflating: Testing/glioma_tumor/image(16).jpg  
  inflating: Testing/glioma_tumor/image(17).jpg  
  inflating: Testing/glioma_tumor/image(18).jpg  
  inflating: Testing/glioma_tumor/image(19).jpg  
  inflating: Testing/glioma_tumor/image(2).jpg  
  inflating: Testing/glioma_tumor/image(20).jpg  
  inflating: Testing/glioma_tumor/image(21).jpg  
  inflating: Testing/glioma_tumor/image(22).jpg  
  inflating: Testing/glioma_tumor/image(23).jpg  
  inflating: Testing/glioma_tumor/image(24).jpg  
  inflating: Testing/glioma_tumor/image(25).jpg  
  in

In [10]:

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
import matplotlib.pyplot as plt

In [14]:
train_path = './Training'
test_path = './Testing'

In [12]:
image_datagen = ImageDataGenerator(
        rescale = 1./255,
        shear_range = 0.2,
        zoom_range = 0.2,
        horizontal_flip = True)

In [15]:
train_generator = image_datagen.flow_from_directory(
        train_path,
        target_size = (64, 64),
        color_mode = 'grayscale',
        batch_size = 1,
        class_mode = 'categorical')

test_generator = image_datagen.flow_from_directory(
        test_path,
        target_size = (64, 64),
        color_mode = 'grayscale',
        batch_size = 1,
        class_mode = 'categorical')

Found 2870 images belonging to 4 classes.
Found 394 images belonging to 4 classes.


In [16]:

classifier = Sequential()

classifier.add(Conv2D(32, (3, 3), activation = 'relu', input_shape = (64, 64, 1), padding = 'same'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

classifier.add(Conv2D(64, (3, 3), activation="relu"))
classifier.add(MaxPooling2D(pool_size = (2, 2))) 

classifier.add(Conv2D(64, (3, 3), activation="relu"))
classifier.add(MaxPooling2D(pool_size = (2, 2))) 

classifier.add(Flatten())
classifier.add(Dense(64, activation = 'relu'))
classifier.add(Dropout(0.2))
classifier.add(Dense(units = 4, activation = 'sigmoid'))

classifier.compile(optimizer = 'adam' ,loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 2304)              0

In [17]:
history = classifier.fit(
        train_generator,
        steps_per_epoch = 50,
        epochs = 150,
   )

Epoch 1/150
50/50 [==============================] - 2s 16ms/step - loss: 0.6094 - accuracy: 0.2359
Epoch 2/150
50/50 [==============================] - 1s 16ms/step - loss: 0.5928 - accuracy: 0.3207
Epoch 3/150
50/50 [==============================] - 1s 15ms/step - loss: 0.5714 - accuracy: 0.2764
Epoch 4/150
50/50 [==============================] - 1s 15ms/step - loss: 0.5639 - accuracy: 0.2346
Epoch 5/150
50/50 [==============================] - 1s 16ms/step - loss: 0.5298 - accuracy: 0.4084
Epoch 6/150
50/50 [==============================] - 1s 16ms/step - loss: 0.5833 - accuracy: 0.4205
Epoch 7/150
50/50 [==============================] - 1s 16ms/step - loss: 0.5160 - accuracy: 0.5600
Epoch 8/150
50/50 [==============================] - 1s 16ms/step - loss: 0.5554 - accuracy: 0.3461
Epoch 9/150
50/50 [==============================] - 1s 17ms/step - loss: 0.5563 - accuracy: 0.2777
Epoch 10/150
50/50 [==============================] - 1s 16ms/step - loss: 0.5642 - accuracy: 0.3485

In [18]:
classifier.evaluate(test_generator)


394/394 [==============================] - 4s 9ms/step - loss: 0.6734 - accuracy: 0.3426


[0.6734367609024048, 0.34263959527015686]

In [ ]:
plt.plot(history.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [21]:
history.history


{'accuracy': [0.3400000035762787,
  0.2800000011920929,
  0.2199999988079071,
  0.30000001192092896,
  0.3799999952316284,
  0.3199999928474426,
  0.47999998927116394,
  0.36000001430511475,
  0.36000001430511475,
  0.3199999928474426,
  0.36000001430511475,
  0.3799999952316284,
  0.41999998688697815,
  0.3799999952316284,
  0.41999998688697815,
  0.5,
  0.4000000059604645,
  0.47999998927116394,
  0.4000000059604645,
  0.4399999976158142,
  0.4000000059604645,
  0.41999998688697815,
  0.47999998927116394,
  0.5600000023841858,
  0.5600000023841858,
  0.46000000834465027,
  0.4000000059604645,
  0.4399999976158142,
  0.5400000214576721,
  0.5400000214576721,
  0.6399999856948853,
  0.46000000834465027,
  0.5199999809265137,
  0.5,
  0.5199999809265137,
  0.47999998927116394,
  0.5799999833106995,
  0.5199999809265137,
  0.6200000047683716,
  0.41999998688697815,
  0.36000001430511475,
  0.6800000071525574,
  0.6200000047683716,
  0.6399999856948853,
  0.6200000047683716,
  0.579999983